In [22]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import random

In [23]:
df_ebay = pd.read_csv('../data/processed/ebay_expanded.csv')

In [24]:
df_agg = pd.read_csv("hf://datasets/lakritidis/product-matching/pricerunner_aggregate.csv")

In [25]:
df_agg

,Product ID,Product Title,Vendor ID,Cluster ID,Cluster Label,Category ID,Category Label
0,1,apple iphone 8 plus 64gb silver,1,1,Apple iPhone 8 Plus 64GB,2612,Mobile Phones
1,2,apple iphone 8 plus 64 gb spacegrau,2,1,Apple iPhone 8 Plus 64GB,2612,Mobile Phones
2,3,apple mq8n2b/a iphone 8 plus 64gb 5.5 12mp sim...,3,1,Apple iPhone 8 Plus 64GB,2612,Mobile Phones
3,4,apple iphone 8 plus 64gb space grey,4,1,Apple iPhone 8 Plus 64GB,2612,Mobile Phones
4,5,apple iphone 8 plus gold 5.5 64gb 4g unlocked ...,5,1,Apple iPhone 8 Plus 64GB,2612,Mobile Phones
...,...,...,...,...,...,...,...
35306,47350,smeg fab28 60cm retro style right hand hinge f...,59,47517,Smeg FAB28 Cream,2623,Fridges
35307,47351,smeg fab28 60cm retro style left hand hinge fr...,59,47518,Smeg FAB28 Red,2623,Fridges
35308,47352,smeg fab28 60cm retro style left hand hinge fr...,59,47519,Smeg FAB28 Pink,2623,Fridges
35309,47355,candy 60cm built under larder fridge cru160nek,125,47524,Candy CRU16.0,2623,Fridges


In [26]:
cluster_dict = df_agg.groupby('Cluster Label').apply(lambda x: x['Product Title'].values).to_dict()
cluster_dict['Apple iPhone 8 Plus 64GB']

/tmp/ipykernel_4754/2237328586.py:1: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  cluster_dict = df_agg.groupby('Cluster Label').apply(lambda x: x['Product Title'].values).to_dict()


array(['apple iphone 8 plus 64gb silver',
       'apple iphone 8 plus 64 gb spacegrau',
       'apple mq8n2b/a iphone 8 plus 64gb 5.5 12mp sim free smartphone in gold',
       'apple iphone 8 plus 64gb space grey',
       'apple iphone 8 plus gold 5.5 64gb 4g unlocked sim free',
       'apple iphone 8 plus gold 5.5 64gb 4g unlocked sim free',
       'apple iphone 8 plus 64 gb space grey',
       'apple iphone 8 plus 64gb space grey',
       'apple iphone 8 plus 64gb space grey',
       'apple iphone 8 plus 64gb space grey',
       'apple iphone 8 plus 5.5 single sim 4g 64gb silver',
       'sim free iphone 8 plus 64gb by apple space grey',
       'apple iphone 8 plus 64gb gold smartphone',
       'apple iphone 8 plus 5.5 single sim 4g 64gb grey',
       'apple iphone 8 plus silver 5.5 64gb 4g unlocked sim free',
       'apple iphone 8 plus 64 gb silver',
       'apple iphone 8 plus 64gb silver unlocked',
       'apple iphone 8 plus 14 cm 5.5 64 gb 12 mp ios 11 silver',
       'iphone 8

In [27]:
corr_clust = {}
corr_clust_set = set()

for key in tqdm(cluster_dict.keys()):
    if key in corr_clust_set:
        continue
    corr_clust_set.add(key)
    for pair in cluster_dict.keys():
        if key != pair and  key.split()[0].lower() == pair.split()[0].lower():
            corr_clust[key] = corr_clust.get(key, [key]) + [pair]
            corr_clust_set.add(pair)

100%|██████████| 12849/12849 [00:00<00:00, 12899.84it/s]


In [55]:
data = []
dict_keys = list(cluster_dict.keys())

# create close pairs
# each pair build from common cluster
for key in cluster_dict.keys():
    if len(cluster_dict[key]) == 1:
        data.append({'query' : key,
                     'word' : cluster_dict[key][0],
                     'label' : 1})
    else:
        sim_word1, sim_word2 = random.sample(list(cluster_dict[key]), 2)
        data.append({'query' : sim_word1,
                    'word' : sim_word2,
                    'label' : 1})
        sim_word1, sim_word2 = random.sample(list(cluster_dict[key]), 2)
        data.append({'query' : key,
                    'word' : sim_word1,
                    'label' : 1})
        data.append({'query' : key,
                    'word' : sim_word2,
                    'label' : 1})
        # sim_word = random.choice(cluster_dict[key])
        # if sim_word != word:
        #     data.append({'query' : word,
        #                  'word' : sim_word,
        #                  'label' : 1})
            
        # sim_word = random.choice(cluster_dict[key])
        # if sim_word != word:
        #     data.append({'query' : word,
        #                  'word' : sim_word,
        #                  'label' : 1})

# create different pairs
for key in corr_clust.keys():
    for ind in range(0, len(corr_clust[key]) - 1, 2):
        data.append({'query' : corr_clust[key][ind],
                     'word' : corr_clust[key][ind + 1],
                     'label' : 0})
        if len(cluster_dict[corr_clust[key][ind]]) >= 2:    
            word1_1, word1_2 = random.sample(list(cluster_dict[corr_clust[key][ind]]), 2)
        else:
            word1_1, word1_2 = cluster_dict[corr_clust[key][ind]][0], cluster_dict[corr_clust[key][ind]][0]
        
        if len(cluster_dict[corr_clust[key][ind + 1]]) >= 2:    
            word2_1, word2_2 = random.sample(list(cluster_dict[corr_clust[key][ind + 1]]), 2)
        else:
            word2_1, word2_2 = cluster_dict[corr_clust[key][ind + 1]][0], cluster_dict[corr_clust[key][ind + 1]][0]
        
        data.append({'query' : word1_1,
                     'word' : word2_1,
                     'label' : 0})
        
        data.append({'query' : word1_2,
                     'word' : word2_2,
                     'label' : 0})
        

In [56]:
processed_df = pd.DataFrame(data[::-1])

In [57]:
processed_df

,query,word,label
0,iceq 95 litre slimline under counter refrigera...,iceq 48l table top mini drinks beer fridge loc...,0
1,iceq 95 litre under counter fridge with ice box,iceq 48 litre table top lockable fridge black,0
2,iceQ IceQ95w White,iceQ Iceq48b Black,0
3,iceq 93 litre under counter glass door display...,iceq 96 litre fridge freezer,0
4,iceq 93 litre under counter glass door display...,iceq 96 litre fridge freezer,0
...,...,...,...
47541,AEG A72700GNX0 Silver,aeg a72700gnx0 60cm freestanding frost free fr...,1
47542,AEG A72020GNW0 White,aeg a72020gnw0/a72020gnw0,1
47543,AEG A72020GNW0 White,aeg a72020gnw0 freezer 60cm a energy rating in...,1
47544,aeg a72020gnw0/a72020gnw0,aeg a72020gnw0 freezer 60cm a energy rating in...,1


In [58]:
processed_df = pd.concat([df_ebay, processed_df], ignore_index=True)

In [59]:
processed_df.groupby('label').count()

,Unnamed: 0,query,word
label,,,
0.0,4248,23265,23265
1.0,4298,32827,32827


In [60]:
processed_df

,Unnamed: 0,query,word,label
0,0.0,Hisense U8N ULED 75 4K Smart TV,"Hisense U8N 75"" ULED TV",0.0
1,1.0,Hisense U8N ULED 75 4K Smart TV,"Hisense U8N 75"" ULED TV",0.0
2,2.0,Ayaneo KUN 64GB RAM 4TB SSD,Alienware m18 R2 Intel i9-14900HX 64GB RAM 4TB...,0.0
3,3.0,Ayaneo KUN 64GB RAM 4TB SSD,Alienware m18 R2 Intel i9-14900HX 64GB RAM 4TB...,0.0
4,4.0,Ayaneo KUN 64GB RAM 4TB SSD,OneXPlayer 2 Pro 64GB RAM 4TB SSD,0.0
...,...,...,...,...
56087,NaN,AEG A72700GNX0 Silver,aeg a72700gnx0 60cm freestanding frost free fr...,1.0
56088,NaN,AEG A72020GNW0 White,aeg a72020gnw0/a72020gnw0,1.0
56089,NaN,AEG A72020GNW0 White,aeg a72020gnw0 freezer 60cm a energy rating in...,1.0
56090,NaN,aeg a72020gnw0/a72020gnw0,aeg a72020gnw0 freezer 60cm a energy rating in...,1.0


In [61]:
processed_df.drop('Unnamed: 0', axis=1, inplace=True)

In [62]:
processed_df.to_csv('../data/processed/combined_data.csv')

In [64]:
token = 'v^1.1#i^1#f^0#I^3#r^0#p^3#t^H4sIAAAAAAAA/+1Za2xbVx2PHSdT1WVMMKCUVTPeUKWha9+37bvalRc7ixsncWP3Feii43vPtc9yH8495yb1KiQ3sDKhwQSbBNO0qEVaGTBtiEndQGJDBXXVhOgnwgcQYlKLkIq0aoAKpVScayeum5U2sYtmCe4X+577f/3+j/M/D7Y+uOnBo6NHLw357vAfq7N1v8/HbWY3DQ587q5+/9aBPraNwHes/kA9sNj/px0YmEZVmYK4alsYBg+ZhoWVxmAi5DqWYgOMsGIBE2KFqEohNZ5T+DCrVB2b2KpthILZdCIUVXktKmiSzqksz7MCHbVWZRbtREgAMB4VQKwki7woSyr9jrELsxYmwCKJEM/yEsPGGFYqcpwiyYooh9m4PB0K7oUORrZFScJsKNkwV2nwOm223txUgDF0CBUSSmZTI4XJVDadmSjuiLTJSq74oUAAcfH1b8O2BoN7geHCm6vBDWql4KoqxDgUSTY1XC9USa0a04H5DVfLepzXS0DV5ZikqiB+W1w5YjsmIDe3wxtBGqM3SBVoEURqt/Io9UbpMaiSlbcJKiKbDno/u11gIB1BJxHKPJw6sKeQmQoFC/m8Y88jDWoeUj4qs4LIczEulMSqA4EpC1FpRUtT1IqP16gZti0NeR7DwQmbPAypyXCtY8Q2x1CiSWvSSenEM6edTmg5kJ/2ItoMoUsqlhdUaFIvBBuvt3b/aj5cy4DblRGaIEm6ShVrUkwGmnSjjPBqfaNZkfQCk8rnI54tsARqjAmcWUiqBlAho1L3uiZ0kKYIks4LMR0ymhzXGTGu60xJ0mSG0yFkISyV1HjsfyY5CHFQySWwlSBrPzQQJkIF1a7CvG0gtRZaS9KYbVbS4RBOhCqEVJVIZGFhIbwghG2nHOFZlovsH88V1Ao0QahFi25NzKBGYqiQcmGkkFqVWnOI5h1VbpVDScHR8sAhtQI0DDqwmrXX2ZZcO/ofQA4biHqgSFX0FsZRGxOodQVNg/NIhTNI6y1kfLPWo4LE8/EYy4pdgTTsMrLGIanYPQYzM57K5rqCRidQQHoLVNvkwoqrkxAvM2xUYdmuwKaq1axpugSUDJjtsVBKAk/XhF3Bq7pur9Wh7nJuacFSCWC6gub1XQUBXSH2LLQ+MJN6tf6hY53KjExlCqMzxcmxzERXaKeg7kBcKXpYey1PU7tTYyn6jOdn8aT9yPj8LsEdU0eGS9PO3IHMaC61O6MDd3gulZsul2yokccNNztGpANOJY2B5kbwXrsckQ2znEh05aQCpOudHpu6auIj5n4dV6P8nkOPk9mFCWEMD0/hwpy7UFgAIlfcZfJgpFgAttgd+PFyr1U67bi3qdsWb1jiLTFerX9YIJ1mYc40ZqEZ+tYV0Ey55+ZrmeO5qCaJHA0jiKl6jBOjOi/yOn2g2GVgvfbbY3jTJqL/auMMcAndz1cQk59K08UHjEslGOMYoOlREbR2VJ325V4L8+1qy9jbvf0XoXm13gE8TwamQkAVhb2VQ1i1zYhNQ1zxhmYaVgfXQxTBdPcXbu73qeQw3WBrtmXUOmHeAA+y5ul+0XZqnShsMW+AB6iq7VqkE3UrrBvg0F1DR4bhHQp0orCNfSNmWsCoEaTijlQiy8s2vAGWKqg1AGoIV716WRcnHTOho8Iw0poHi50Y60CqEDRO0jph2qDKlsmWTZCO1KYM7Jaw6qDq+q1Qba/WbyWrE39gWgsbCl2TYV2q2rigBg00D9dbdi2/URa7ux081JADVTLjOqi3usxKc50ZB/SdWdNqmTnNtGdxCXcF3vNqLx7MjBzId4UrDed7bakEBV3gY9EYE5VhlBH1uMoAKGqMFGXpDxSjArvu44vAov/tG+LuucMoLiqJQlzqdk8zBYFh9hayqmNrrurNqv9Htmag7dLiA5dVkeuvipN9jYdb9J1iF31v+n0+dgf7We5+9jOD/XsC/XduxYjQfg70MEZlCxDXgeFZWKsC5Pg/1nf2rpx2ZDT3t3rJfX3fX3fG+obabqqPHWS3tO6qN/Vzm9surtl7r30Z4D7yySFeYmOsxHGSLMrT7P3Xvga4TwTu+fqPr2xevnzifPHwP9l9539VH3zjofvYoRaRzzfQF1j09SXCR74U/vnSpPj9f+w7+NYr77yAxNr03U/tKriztrPtQn77ye/+5PnM2z+Uz5xefvf98+9fMo4vpZ/ZueXwH4Zzl1/68/CjkWe1i198ceniMj7sf+zzrwwdzvoHLgbx3h+dGT3Yt/R3f+0hY9enp8VLV7a8d2U5EjtOLpz69ZO5ysm072rfM9rxbzhHvnX6N2NPD+5/YiCdfPTq1q998+NHv3fkqS+kT/6+7n/3O+q5Tw299tE701ff+eW5P5r3cr99Kzf3ct/PrNM/Lf3CePnpH7yeuHzuiW0P7kRnX5x789LSV558NvBC/dWX7jj37fcGTgVqu9HZqyfOzNQvLDP38du2S1+t/uXuSDjxwO+07V9+buBfJwLNWP4bnzZTFkMgAAA='

In [ ]:
import requests

item = 'iphone 16 256gb'
url = f'https://api.ebay.com/buy/browse/v1/item_summary/search?q={item}'


headers = {
    "Authorization": f"Bearer {token}",
    "Content-Type": "application/json"
}

params = {
    "q": item,
    "limit": 10, 
    "buyingOptions": "FIXED_PRICE"  
}

response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    print(response.text)

    items = response.json().get('itemSummaries', [])
    print(f'Получено товаров: {len(items)}')
    for i, item_info in enumerate(items, 1):
        print(f"{i}. {item_info.get('title')} - {item_info.get('price')['value']}")
else:
    print("Ошибка:", response.text)

{"href":"https://api.ebay.com/buy/browse/v1/item_summary/search?q=iphone+16+256gb&limit=10&offset=0","total":12551,"next":"https://api.ebay.com/buy/browse/v1/item_summary/search?q=iphone+16+256gb&limit=10&offset=10","limit":10,"offset":0,"itemSummaries":[{"itemId":"v1|296819057702|594602532244","title":"Apple iPhone 16 256GB Factory Unlocked AT&T T-Mobile Verizon Very Good Condition","itemGroupHref":"https://api.ebay.com/buy/browse/v1/item/get_items_by_item_group?item_group_id=296819057702","leafCategoryIds":["9355"],"categories":[{"categoryId":"9355","categoryName":"Cell Phones & Smartphones"},{"categoryId":"15032","categoryName":"Cell Phones & Accessories"}],"image":{"imageUrl":"https://i.ebayimg.com/images/g/v8QAAOSwufdnN3K4/s-l225.jpg"},"price":{"value":"676.95","currency":"USD"},"itemGroupType":"SELLER_DEFINED_VARIATIONS","itemHref":"https://api.ebay.com/buy/browse/v1/item/v1%7C296819057702%7C594602532244","seller":{"username":"supplytronics","feedbackPercentage":"98.3","feedbackS

In [66]:
df_row_labels = pd.read_csv('../data/raw/from_ebay_labeled.csv')